In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import openai
from openai import OpenAI
import time
from tqdm import tqdm

import importlib
import sys; sys.path.append("../src")
import emotion
importlib.reload(emotion)
from emotion import EmotionExample, get_llm_generated_answer, isolate_individual_features, distill_relevant_features, calculate_expert_alignment_score, load_emotion_data, run_pipeline

### Load Emotion Data

In [ ]:
emotion_data =  load_dataset("BrachioLab/emotion")
emotion_data = emotion_data['train'].to_pandas()
emotion_data = emotion_data.sample(3, random_state=11).reset_index(drop=True)

emotion_labels = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral"
}


### Stage 0: Get LLM Explanations


In [ ]:
emotion_examples = []
for idx,row in tqdm(emotion_data.iterrows()):
    llm_label, explanation = get_llm_generated_answer(row['text'])
    if llm_label is None:
        continue
    emotion_examples.append(EmotionExample(
        text=row['text'],
        ground_truth=emotion_labels[row['labels'][0]],
        llm_label=llm_label,
        llm_explanation=explanation
    ))

In [ ]:
[emotion_examples[i].llm_label for i in range(len(emotion_examples))]

In [ ]:
emotion_examples[0].llm_explanation

### Stage 1: Atomic claim extraction

In [ ]:
for example in emotion_examples:
    claims = isolate_individual_features(example.llm_explanation)
    if claims is None:
        continue
    example.claims = [claim.strip() for claim in claims]

In [ ]:
emotion_examples[0].claims

### Stage 2: Distill relevant claims

In [ ]:
for example in emotion_examples:
    relevant_claims = distill_relevant_features(example)
    example.relevant_claims = relevant_claims

In [ ]:
emotion_examples[0].relevant_claims

### Stage 3: Calculate alignment scores


In [ ]:
for example in emotion_examples:
    alignment_scores = []
    alignment_categories = []
    reasonings = []
    for claim in tqdm(example.relevant_claims):
        category, alignment_score, reasoning = calculate_expert_alignment_score(claim)
        if category is None:
            continue
        alignment_scores.append(alignment_score)
        alignment_categories.append(category)
    example.alignment_scores = alignment_scores
    example.alignment_categories = alignment_categories
    example.final_alignment = np.mean(alignment_scores)

In [ ]:
emotion_examples[0].alignment_scores

In [ ]:
emotion_examples[0].alignment_categories